# Content analysis

In [85]:
from bs4 import BeautifulSoup
from colorama import Fore, Back, Style
import csv
import os
import pandas as pd
import re
import sys
import przona

dummy = csv.field_size_limit(sys.maxsize)

In [2]:
CSV_DIR = "csv/"
RECOMMENDATIONS_FILE = "recommendation_web_pages.csv"

web_pages = przona.read_dict(CSV_DIR+RECOMMENDATIONS_FILE, spy=True)

155051


## Text segmentation

In [53]:
def get_paragraphs(soup):
    paragraphs = []
    for paragraph in re.sub('\\\\r',' ',re.sub('\\\\t',' ',soup.text)).split('\\n'):
        paragraph = re.sub("\s+", " ", paragraph).strip()
        if paragraph != "":
            paragraphs.append(paragraph)
    return(paragraphs)

In [54]:
if os.path.isfile(CSV_DIR+"paragraphs.csv"):
    paragraphs = przona.read_dict(CSV_DIR+"paragraphs.csv")
else:
    paragraphs = {}
    counter = 0
    for url in web_pages.keys():
        counter += 1
        if counter % 100 == 0:
            squeal(counter)
        web_page_text = web_pages[url][0]
        web_page_text_with_spaces = re.sub(">", "> ", web_page_text)
        soup = BeautifulSoup(web_page_text_with_spaces)
        paragraphs[url] = get_paragraphs(soup)
    przona.save_dict(paragraphs, CSV_DIR+"paragraphs.csv")

## Keyword search

In [3]:
KEYWORD_FILE = "../data/beeldvormende_diagnostiek.txt"

In [4]:
paragraphs = przona.read_dict(CSV_DIR+"paragraphs.csv")

In [5]:
infile = open(KEYWORD_FILE, "r")
keywords = []
for line in infile:
    keywords.append(line.strip().lower())
infile.close()
len(keywords)

113

In [51]:
if os.path.isfile(CSV_DIR+"matches.csv"):
    matches = przona.read_dict(CSV_DIR+"matches.csv")
else:
    matches = {}
    counter = 0
    for url in paragraphs:
        text = " ".join(paragraphs[url])
        matches[url] = []
        for keyword in keywords:
            if re.search(r'\b'+keyword+r'\b', text, flags=re.IGNORECASE):
                matches[url].append(keyword)
        counter += 1
        if counter % 100 == 0:
            przona.squeal(counter)
    przona.squeal(counter)
    przona.save_dict(matches, CSV_DIR+"matches.csv")

In [52]:
counter = 0
for url in sorted(matches.keys(), key=lambda url:len(matches[url]), reverse=True):
    print(len(matches[url]), len((" ".join(paragraphs[url])).lower()), url)
    counter += 1
    if counter >= 20:
        break

21 48293 /richtlijn/meningeoom/diagnostiek/beeldvorming.html
19 102977 /richtlijn/diabetische_voet/diagnostiek_en_behandeling_charcot/diagnostiek_van_acute_charcot-voet.html
19 45354 /richtlijn/necrotiserende_wekedeleninfecties/beeldvormende_diagnostiek_nwdi.html
18 90994 /richtlijn/klinische_postmortem_radiologie/indicatiestelling_bij_foetussen_en_neonaten.html
17 43144 /richtlijn/fractuur-gerelateerde_infecties_fri_s/beeldvormend_onderzoek.html
17 40131 /richtlijn/veneuze_pathologie_varices/varices-diagnostiek_en_onderzoek/varices-overige_beeldvormende_technieken.html
17 52384 /richtlijn/primaire_tumor_onbekend/diagnostiek/beeldvormend_onderzoek.html
17 27431 /richtlijn/psa_bij_kinderen_op_locaties_buiten_de_ok/randvoorwaarden_toedienen_psa_bij_kinderen/procedures_voor_psa_bij_kinderen.html
16 106091 /richtlijn/borstkanker/screening/screening_binnen_het_bob/bob.html
16 108900 /richtlijn/borstkanker/diagnostiek/preoperatieve_stadiering/pet_ct.html
16 67006 /richtlijn/schildkliercarcin

In [49]:
keyword = keywords[0]
url = "/richtlijn/meningeoom/diagnostiek/beeldvorming.html"
for p in paragraphs[url]:
    matches_per_p = []
    for keyword in keywords:
        if re.search(keyword, p, flags=re.IGNORECASE):
            p = re.sub(r'(\b'+keyword+r'\b)', Style.BRIGHT+"\\1"+Style.RESET_ALL, p, flags=re.IGNORECASE)
            matches_per_p.append(keyword)
    if len(set(matches_per_p)) >= 5: 
        print(Fore.GREEN+Style.BRIGHT+str(len(set(matches_per_p))), str(matches_per_p)+Style.RESET_ALL, p)

6 ['beeld', 'ct', 'beeldvorming', 'mr', 'mri', 'scan'] Indien beeldvorming van een (vermoedelijk) meningeoom door MRI bij de diagnostiek of follow-up niet haalbaar is, bijvoorbeeld doordat een patiënt niet lang plat kan liggen of bij contra-indicaties voor een MRI, is een CT-scan, bij voorkeur zonder en met jodiumhoudend contrast, een redelijk alternatief. Daar staat tegenover dat follow-up van een meningeoom met CT een herhaalde stralenbelasting betekent; dit in tegenstelling tot MRI.
9 ['beeld', 'computer', 'ct', 'tomografie', 'beeldvormende', 'mr', 'mri', 'radiologisch', 'scan'] Voor het vaststellen van de aanwezigheid van een ruimte-innemende intracraniële afwijking, de nadere differentiatie hiervan en/of de follow-up staan meerdere beeldvormende modaliteiten ter beschikking. Als eerste modaliteit voor patiënten met acute symptomatologie wordt vaak gebruik gemaakt van een blanco CT-scan, eventueel aangevuld met jodiumhoudend contrast ( 1 ). Voor het vaststellen of er sprake is van 

## Analysis based on treatment phase

In [72]:
defined_phases = ["diagnostiek", "behandeling", "therapie", "preventie", "interventie", "onderzoek", "nazorg", "screening"]

In [66]:
phases = {}
for url in web_pages:
    if re.search("^/richtlijn/", url):
        try:
            phase = url.split("/")[3]
            if phase != "":
                for token in phase.split("_"):
                    if token in phases: 
                        phases[token] += 1
                    else:
                        phases[token] = 1
        except:
            pass
list({phase:phases[phase] for phase in sorted(phases.keys(), key=lambda phase:phases[phase], reverse=True)}.items())[:10]

[('bij', 2871),
 ('behandeling', 1530),
 ('van', 1177),
 ('en', 1123),
 ('diagnostiek', 787),
 ('-', 614),
 ('zorg', 361),
 ('organisatie', 280),
 ('de', 276),
 ('acute', 243)]

In [77]:
counter = 0
analysis = {}
for url in paragraphs:
    if re.search("/richtlijn", url):
        text = " ".join(paragraphs[url])
        topic = url.split("/")[2]
        matches_page = None
        for phase in defined_phases:
            if re.search(phase, url):
                if matches_page == None:
                    matches_page = []
                    for keyword in keywords:
                        if re.search(r'\b'+keyword+r'\b', text, flags=re.IGNORECASE):
                            matches_page.append(keyword)
                if topic not in analysis:
                    analysis[topic] = {}
                analysis[topic][phase] = len(matches_page)
    counter += 1
    if counter % 100 == 0:
        przona.squeal(counter)
przona.squeal(counter)

47689


In [78]:
len(analysis)

323

In [90]:
total = {}
for topic in analysis:
    total_row = 0
    for phase in analysis[topic]:
        if phase not in total: total[phase] = 0
        total[phase] += analysis[topic][phase]
        total_row += analysis[topic][phase]
    analysis[topic]["totaal"] = total_row
analysis["totaal"] = total

In [106]:
analysis = {topic:{ phase:analysis[topic][phase] for phase in sorted(analysis[topic].keys(), key=lambda phase:analysis["totaal"][phase], reverse=True)}
            for topic in sorted(analysis.keys(), key=lambda t:analysis[t]["totaal"], reverse=True)}

In [107]:
pd.DataFrame(analysis).T

,totaal,diagnostiek,behandeling,onderzoek,therapie,screening,interventie,preventie,nazorg
totaal,3368.0,933.0,725.0,569.0,452.0,258.0,163.0,159.0,109.0
colorectaal_carcinoom_crc,71.0,14.0,8.0,8.0,9.0,8.0,8.0,8.0,8.0
borstkanker,68.0,16.0,9.0,12.0,11.0,11.0,NaN,NaN,9.0
acute_neurologie,67.0,11.0,11.0,11.0,7.0,12.0,9.0,6.0,NaN
dementie,59.0,10.0,10.0,10.0,10.0,9.0,10.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
myotone_dystrofie_type1,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
palliatieve_zorg_voor_mensen_met_copd,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
hormoontherapie_van_klachten_in_het_climacterium_en_de_postmenopauze,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
hemorrhagia_postpartum_hpp,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
